In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("ticks")
%matplotlib qt
# you can use '%matplotlib inline' for:  plots will open in IDE   

In [42]:
# reading files 
df = pd.read_csv( "logs.csv", delimiter=" ")
data = {tag: df.loc[df.tag == tag] for tag in set(df.tag)}
print('All possible tags + count of images ( the count should be equal 536):\n', *[(tag, len(data[tag])) for tag in data.keys()])
df.head()

# check all the tags
# most_actual_tags = [tag for tag in sorted(data, reverse=True)]

# use only most actual tags    ** have to be unique tags
most_actual_tags = ['origin', 'w_filters_d32', 'morf_c&o_d32', 'morf_c_d32', 'w_filters_m20']

All possible tags + count of images ( the count should be equal 536):
 ('morf_c&o_d32', 536) ('origin', 536) ('w_filters_d8', 536) ('morf_o_d32', 536) ('morf_c_d32', 536) ('w_filters_m20', 536) ('w_filters_d32', 536)


In [38]:
# total count of contours after identify candidates ( distribution )
plt.figure(figsize=(4 * len(most_actual_tags),6))
for i, tag in enumerate(most_actual_tags):
    
    plt.subplot(1, len(most_actual_tags), i+1)
    plt.title(tag)
    sns.scatterplot(x= (data[tag].index - data[tag].index[0]), y=data[tag].total,color='g')


In [39]:
# distribution of the number of the total number of contours

plt.figure(figsize=(4*len(most_actual_tags),10))
for i, tag in enumerate(most_actual_tags):
    
    total_count = data[tag]['total'].value_counts() 
    plt.subplot(len(most_actual_tags) // 4 + (len(most_actual_tags) % 4), len(data) // 2 + (len(data) % 2), i+1)
    plt.title(tag)
    
    sns.scatterplot(total_count, color='g')


In [43]:
# count of the total number of contours

plt.figure(figsize=(4*len(most_actual_tags),10))
means = pd.DataFrame()

for i, tag in enumerate(most_actual_tags):
   
    data[tag] = data[tag].rename({f'depth{i}': f'{i}' for i in range(1,9)}, axis=1).drop(['tag'], axis=1)
    means[tag] = data[tag].mean().rename('mean')

    plt.subplot(2, len(data)// 2 + len(data) % 2, i+1)
    plt.title(tag)
    sns.scatterplot(means[tag], color='g')
    


In [16]:
# average values for each tag and depth
means.head()

,w_filters_m20,w_filters_d8,w_filters_d32,origin,morf_o_d32,morf_c_d32,morf_c&o_d32
1,91.595149,91.595149,91.595149,91.595149,204.289179,174.901119,287.595149
2,72.986940,72.986940,72.986940,NaN,138.399254,146.569030,211.981343
3,55.097015,55.097015,55.097015,NaN,88.001866,108.160448,141.065299
4,42.345794,NaN,42.345794,NaN,64.654206,104.672897,126.981308
5,29.555102,NaN,29.494908,NaN,41.920570,63.077393,75.503055


In [45]:
# box-plots for each tag and depth
# ** you can use a zoom in matplotlib window 
for i, tag in enumerate(most_actual_tags):
    
    temp_df = data[tag]
    plt.figure(figsize=(15, 5))
    plt.suptitle(f'{tag}')
      
    for depth in range(1, 9):
        
        # skip empty depth
        if data[tag][f'{depth}'].isnull().all():
            continue    
    
        plt.subplot(2, 4, depth)
        sns.boxplot(y=f'{depth}', data=temp_df)
        plt.xlabel(f'Depth: {depth}')  
        plt.ylabel('Count of new_contours')
    
    plt.show()
    

In [52]:
# check_benchmark
path = 'benchmark\\'

benchmark_results = pd.DataFrame({'tag': most_actual_tags, 
                                  'detect': [None] * len(most_actual_tags), 
                                  'decode':  [None] * len(most_actual_tags), 
                                  'time':  [None] * len(most_actual_tags)})
for tag in most_actual_tags:
    curr_path = path + tag + '.txt'
    file = open(curr_path)
    
    detect, decode, time = float(file.readline().split()[1]), float(file.readline().split()[1]), float(file.readline())
    
    benchmark_results.loc[benchmark_results['tag'] == tag, 'detect'] = detect
    benchmark_results.loc[benchmark_results['tag'] == tag, 'decode'] = decode
    benchmark_results.loc[benchmark_results['tag'] == tag, 'time'] = time
    
benchmark_results.head()

,tag,detect,decode,time
0,origin,0.796266,0.587662,186.811594
1,w_filters_d32,0.862825,0.616071,218.997977
2,morf_c&o_d32,0.868506,0.625,251.534142
3,morf_c_d32,0.866883,0.627435,232.602334
4,w_filters_m20,0.862825,0.61526,211.730188


In [53]:
fig, ax = plt.subplots(figsize=(15, 8))

colors = ['skyblue', 'lightgreen']
bar_width = 0.1

# creating bar_plot to compare detect and decode
x = np.arange(len(most_actual_tags))
for i, col in enumerate(['detect', 'decode']):
    y = benchmark_results[col].tolist()
    ax.bar(x + i*bar_width, y, bar_width, label=col, color=colors[i])

# some plot settings
ax.set_xticks(x + bar_width)
ax.set_xticklabels(most_actual_tags)
ax.set_xlabel('Tags')
ax.set_ylabel('Values')
ax.set_title('Comparison of Tags by Columns')
ax.legend()



In [54]:
# bar plot for 'time' 
fig, ax = plt.subplots(figsize=(10, 8))
plt.subplot(1, 1, 1)
plt.bar(x,benchmark_results['time'].tolist(), bar_width, label='time', color='skyblue')

ax.set_xticks(x + bar_width)
ax.set_xticklabels(most_actual_tags)
ax.set_xlabel('Tags')
ax.set_ylabel('Values')
ax.set_title('Comparison of Tags by time')
ax.legend()

plt.show()